In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import qmc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def assure_path_exists(path):
    if not os.path.exists(path):
        print(f"Creating directory: {path}")
        os.makedirs(path)
    else:
        print(f"Directory already exists: {path}")

def lhs_params(param_ranges, n_samples):
    n_params = len(param_ranges)
    sampler = qmc.LatinHypercube(d=n_params)
    lhs_samples = sampler.random(n=n_samples)
    for i in range(n_params):
        min_val, max_val = param_ranges[i]
        lhs_samples[:, i] = min_val + (max_val - min_val) * lhs_samples[:, i]
    return lhs_samples

def ComponentRescaling(sites, HistoricalMonthlyQ, LHsamples, rescaled_folder):
    mMonths = 12
    nScenarios = LHsamples.shape[0]
    nYears = len(HistoricalMonthlyQ) // mMonths

    historical_avg = np.mean(HistoricalMonthlyQ.reshape(nYears, mMonths), axis=0)
    
    for j in range(nScenarios):
        amplitude, lambda_coeff = LHsamples[j, :]
        
        NewMonthlyQ = np.zeros(len(HistoricalMonthlyQ))
        for m in range(12):
            k = (m + 3) % 12
            q_new = amplitude * (lambda_coeff * historical_avg[k] + (1 - lambda_coeff) * historical_avg[m])
            NewMonthlyQ[m::mMonths] = q_new
        
        for site in sites:
            file_path = os.path.join(rescaled_folder, f"{site}_Scenario{j+1}.csv")
            np.savetxt(file_path, NewMonthlyQ, fmt='%.3f', delimiter=',')
        
        scaling_factors = NewMonthlyQ[:mMonths] / historical_avg
        scaling_factors_path = os.path.join(rescaled_folder, f"Scenario{j+1}_scaling_factors.txt")
        np.savetxt(scaling_factors_path, scaling_factors, fmt='%.3f')

def CombineRescaledWithRainbase(sites, nScenarios, rescaled_folder, rainbase_data, combined_folder):
    nYears = 30
    mMonths = 12
    total_months = nYears * mMonths
    
    if len(rainbase_data) < total_months:
        rainbase_data = np.tile(rainbase_data, (total_months // len(rainbase_data) + 1))[:total_months]
    
    rainbase_reshaped = rainbase_data.reshape(nYears, mMonths)
    
    for i in range(nScenarios):
        for site in sites:
            rescaled_data = np.loadtxt(os.path.join(rescaled_folder, f"{site}_Scenario{i+1}.csv"), delimiter=',')
            
            if rescaled_data.shape != (nYears, mMonths):
                rescaled_data = rescaled_data.reshape(nYears, mMonths)
            
            combined_data = rescaled_data + rainbase_reshaped
            file_path = os.path.join(combined_folder, f"total_{site}_Scenario{i+1}.csv")
            np.savetxt(file_path, combined_data, fmt='%.3f', delimiter=',')

def CombineSupplyFlows(sites, nScenarios, combined_folder, supply_flows_rescaled_folder):
    nYears = 30
    mMonths = 12
    for i in range(1, nScenarios + 1):
        combined_sum = np.zeros((nYears, mMonths))
        
        for site in sites:
            file_path = os.path.join(combined_folder, f"total_{site}_Scenario{i}.csv")
            data = np.loadtxt(file_path, delimiter=',')
            combined_sum += data
        
        output_file_path = os.path.join(supply_flows_rescaled_folder, f"supply_Scenario{i}.csv")
        np.savetxt(output_file_path, combined_sum, fmt='%.3f', delimiter=',')
        print(f"Saved combined supply flow to {output_file_path}")

def calculate_average_flows(input_folder, output_folder, prefix):
    assure_path_exists(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            data = np.loadtxt(file_path, delimiter=',')
            
            average_data = np.mean(data, axis=0)
            
            output_file_path = os.path.join(output_folder, f"{prefix}_{filename}")
            np.savetxt(output_file_path, average_data.reshape(1, -1), fmt='%.3f', delimiter=',')

def plot_rescaled_combined_flows_against_historical_mean(supply_flows_rescaled_FINAL, historical_data, results_folder):
    assure_path_exists(results_folder)
    
    historical_mean = np.mean(historical_data.reshape(-1, 12), axis=0)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    months = range(1, 13)
    
    ax.plot(months, historical_mean, label='Historical Mean', color='black', linewidth=2)

    for file in os.listdir(supply_flows_rescaled_FINAL):
        if file.endswith('.csv'):
            scenario_data = np.loadtxt(os.path.join(supply_flows_rescaled_FINAL, file), delimiter=',')
            scenario_mean = np.mean(scenario_data.reshape(-1, 12), axis=0)
            ax.plot(months, scenario_mean, label=file.split('.')[0], alpha=0.5)

    ax.set_title('Rescaled Combined Flows vs Historical Mean')
    ax.set_xlabel('Month')
    ax.set_ylabel('Water Volume')
    ax.set_xticks(months)
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.legend(loc='upper right')

    plt.tight_layout()
    plot_path = os.path.join(results_folder, 'rescaled_combined_flows_vs_historical_mean.png')
    plt.savefig(plot_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Plot saved to {plot_path}")

def plot_individual_rescaled_flows(supply_flows_rescaled_FINAL, historical_data, results_folder, lhs_samples):
    assure_path_exists(results_folder)
    
    historical_mean = np.mean(historical_data.reshape(-1, 12), axis=0)
    
    months = range(1, 13)
    
    for i, file in enumerate(sorted(os.listdir(supply_flows_rescaled_FINAL))):
        if file.endswith('.csv'):
            scenario_data = np.loadtxt(os.path.join(supply_flows_rescaled_FINAL, file), delimiter=',')
            scenario_mean = np.mean(scenario_data.reshape(-1, 12), axis=0)
            amplitude, lambda_coeff = lhs_samples[i]

            fig, ax = plt.subplots(figsize=(12, 6))
            
            ax.plot(months, historical_mean, label='Historical Mean', color='black', linewidth=2)
            
            ax.plot(months, scenario_mean, label=f'Rescaled {i+1}', alpha=0.5, color='blue')

            ax.set_title(f'Scenario {i+1}: Rescaled Combined Flows vs Historical Mean\nAmplitude: {amplitude:.3f}, Lambda: {lambda_coeff:.3f}')
            ax.set_xlabel('Month')
            ax.set_ylabel('Water Volume')
            ax.set_xticks(months)
            ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
            ax.legend(loc='upper right')

            plot_path = os.path.join(results_folder, f'scenario_{i+1}_rescaled_combined_flows_vs_historical_mean.png')
            plt.tight_layout()
            plt.savefig(plot_path, bbox_inches='tight', dpi=100)
            plt.close()
            print(f"Plot for Scenario {i+1} saved to {plot_path}")


# def plot_combined_ranges(historical_data, rescaled_folder, results_folder):
#     assure_path_exists(results_folder)
    
#     historical_data = historical_data.reshape(-1, 12)
#     historical_mean = np.mean(historical_data, axis=0)
#     historical_min = np.min(historical_data, axis=0)
#     historical_max = np.max(historical_data, axis=0)

#     rescaled_data = []
#     for file in os.listdir(rescaled_folder):
#         if file.endswith('.csv'):
#             data = np.loadtxt(os.path.join(rescaled_folder, file), delimiter=',')
#             rescaled_data.append(data)
    
#     rescaled_data = np.array(rescaled_data)
#     rescaled_mean = np.mean(rescaled_data, axis=0)
#     rescaled_min = np.min(rescaled_data, axis=0)
#     rescaled_max = np.max(rescaled_data, axis=0)

#     # Calculate average of all historical scenarios
#     historical_avg = np.mean(historical_data)

#     # Calculate average of all rescaled scenarios
#     rescaled_avg = np.mean(rescaled_data)

#     months = range(1, 13)
#     fig, ax = plt.subplots(figsize=(12, 6))

#     ax.fill_between(months, historical_min, historical_max, alpha=0.2, color='black')
#     ax.plot(months, historical_mean, color='black', linewidth=2, label='Historical Supply')
#     ax.axhline(y=historical_avg, color='black', linestyle='--', label='Avg. Historical Supply')



#     ax.fill_between(months, rescaled_min, rescaled_max, alpha=0.2, color='red')
#     ax.plot(months, rescaled_mean, color='red', linewidth=2, label='Paramterised Supply')
#     ax.axhline(y=rescaled_avg, color='red', linestyle='--', label='Avg. Paramterised Supply')



#     ax.set_title('Combined Range: Historical vs Rescaled Flows')
#     ax.set_xlabel('Month')
#     ax.set_ylabel('Streamflow $km^3$')
#     ax.set_xticks(months)
#     ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
#     ax.legend(loc='upper right')

#     plt.tight_layout()
#     plot_path = os.path.join(results_folder, 'combined_range_historical_vs_rescaled.png')
#     plt.savefig(plot_path, bbox_inches='tight', dpi=300)
#     plt.close()
#     print(f"Plot saved to {plot_path}")

def plot_combined_ranges(historical_data, rescaled_folder, results_folder):
    assure_path_exists(results_folder)
    
    historical_data = historical_data.reshape(-1, 12)
    historical_mean = np.mean(historical_data, axis=0)
    historical_min = np.min(historical_data, axis=0)
    historical_max = np.max(historical_data, axis=0)

    rescaled_data = []
    for file in os.listdir(rescaled_folder):
        if file.endswith('.csv'):
            data = np.loadtxt(os.path.join(rescaled_folder, file), delimiter=',')
            rescaled_data.append(data)
    
    rescaled_data = np.array(rescaled_data)
    rescaled_data = rescaled_data.reshape(rescaled_data.shape[0], -1, 12)  # Reshape to (n_scenarios, n_years, 12)
    rescaled_mean = np.mean(rescaled_data, axis=(0, 1))  # Average across scenarios and years
    rescaled_min = np.min(rescaled_data, axis=(0, 1))
    rescaled_max = np.max(rescaled_data, axis=(0, 1))

    # Calculate average of all historical scenarios
    historical_avg = np.mean(historical_mean)
    # Calculate average of all rescaled scenarios
    rescaled_avg = np.mean(rescaled_mean)

    months = range(1, 13)
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.fill_between(months, historical_min, historical_max, alpha=0.2, color='black')
    ax.plot(months, historical_mean, color='black', linewidth=2, label='Avg. Historical Supply')


    ax.fill_between(months, rescaled_min, rescaled_max, alpha=0.2, color='red')
    ax.plot(months, rescaled_mean, color='red', linewidth=2, label='Avg. Parameterised Supply')


    ax.set_title('Combined Range: Historical vs Rescaled Flows')
    ax.set_xlabel('Month')
    ax.set_ylabel('Streamflow $km^3$')
    ax.set_xticks(months)
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.legend(loc='upper right')

    plt.tight_layout()
    plot_path = os.path.join(results_folder, 'combined_range_historical_vs_rescaled.png')
    plt.savefig(plot_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Plot saved to {plot_path}")


def main():
    sites = ['All']  
    nMonths = 12
    
    all_param_ranges = [
        [0.47, 1.78],  # Amplitude
        [0, 1]        # lambda
    ]
    
    n_samples = 1000

    all_lhs_samples = lhs_params(all_param_ranges, n_samples)
    np.savetxt("LHsamples_all_params.txt", all_lhs_samples, fmt='%.6f')
    
    global supply_lhs_samples
    supply_lhs_samples = all_lhs_samples

    Qgs = np.loadtxt('meltwater_hist_km3.txt')
    Qall = np.loadtxt('total_km3.txt')
    rainbase_data = np.loadtxt('rainbase_km3.txt')

    hist_data = np.loadtxt('meltwater_hist_km3.txt')
    scenario_data = np.loadtxt('total_km3.txt')
    scenario_type = 'Scenario'

    supply_flows_folder = './Supply_flows_plotting'
    rescaled_folder = os.path.join(supply_flows_folder, '1. rescaled_meltwater')
    combined_folder = os.path.join(supply_flows_folder, '2. combined')
    supply_flows_rescaled_folder = os.path.join(supply_flows_folder, '3. supply_flows_rescaled_combined')
    supply_flows_rescaled_FINAL = os.path.join(supply_flows_folder, '4. supply_flows_rescaled_FINAL')
    results_folder = os.path.join(supply_flows_folder, '6. results')
    individual_plots_folder = os.path.join(supply_flows_folder, '7. individual_scenario_plots')
    combined_range_folder = os.path.join(supply_flows_folder, '8. combined_range_plot')
    
    for folder in [supply_flows_folder, rescaled_folder, combined_folder, supply_flows_rescaled_folder, 
                   supply_flows_rescaled_FINAL, results_folder, individual_plots_folder, combined_range_folder]:
        assure_path_exists(folder)

    ComponentRescaling(sites, Qgs, supply_lhs_samples, rescaled_folder)
    CombineRescaledWithRainbase(sites, n_samples, rescaled_folder, rainbase_data, combined_folder)
    CombineSupplyFlows(sites, n_samples, combined_folder, supply_flows_rescaled_folder)
    calculate_average_flows(supply_flows_rescaled_folder, supply_flows_rescaled_FINAL, "FINAL_combined_rescaled")

    plot_individual_rescaled_flows(supply_flows_rescaled_FINAL, Qall, individual_plots_folder, supply_lhs_samples)
    plot_combined_ranges(Qall, supply_flows_rescaled_FINAL, combined_range_folder)

    print("Process completed successfully!")

if __name__ == "__main__":
    main()

Creating directory: ./Supply_flows_plotting
Creating directory: ./Supply_flows_plotting\1. rescaled_meltwater
Creating directory: ./Supply_flows_plotting\2. combined
Creating directory: ./Supply_flows_plotting\3. supply_flows_rescaled_combined
Creating directory: ./Supply_flows_plotting\4. supply_flows_rescaled_FINAL
Creating directory: ./Supply_flows_plotting\6. results
Creating directory: ./Supply_flows_plotting\7. individual_scenario_plots
Creating directory: ./Supply_flows_plotting\8. combined_range_plot
Saved combined supply flow to ./Supply_flows_plotting\3. supply_flows_rescaled_combined\supply_Scenario1.csv
Saved combined supply flow to ./Supply_flows_plotting\3. supply_flows_rescaled_combined\supply_Scenario2.csv
Saved combined supply flow to ./Supply_flows_plotting\3. supply_flows_rescaled_combined\supply_Scenario3.csv
Saved combined supply flow to ./Supply_flows_plotting\3. supply_flows_rescaled_combined\supply_Scenario4.csv
Saved combined supply flow to ./Supply_flows_plott

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import qmc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def assure_path_exists(path):
    if not os.path.exists(path):
        print(f"Creating directory: {path}")
        os.makedirs(path)
    else:
        print(f"Directory already exists: {path}")

def lhs_params(param_ranges, n_samples):
    n_params = len(param_ranges)
    sampler = qmc.LatinHypercube(d=n_params)
    lhs_samples = sampler.random(n=n_samples)
    for i in range(n_params):
        min_val, max_val = param_ranges[i]
        lhs_samples[:, i] = min_val + (max_val - min_val) * lhs_samples[:, i]
    return lhs_samples

def ComponentRescaling(sites, HistoricalMonthlyQ, LHsamples, rescaled_folder):
    mMonths = 12
    nScenarios = LHsamples.shape[0]
    nYears = len(HistoricalMonthlyQ) // mMonths

    historical_avg = np.mean(HistoricalMonthlyQ.reshape(nYears, mMonths), axis=0)
    
    for j in range(nScenarios):
        amplitude, lambda_coeff = LHsamples[j, :]
        
        NewMonthlyQ = np.zeros(len(HistoricalMonthlyQ))
        for m in range(12):
            k = (m + 3) % 12
            q_new = amplitude * (lambda_coeff * historical_avg[k] + (1 - lambda_coeff) * historical_avg[m])
            NewMonthlyQ[m::mMonths] = q_new
        
        for site in sites:
            file_path = os.path.join(rescaled_folder, f"{site}_Scenario{j+1}.csv")
            np.savetxt(file_path, NewMonthlyQ, fmt='%.3f', delimiter=',')
        
        scaling_factors = NewMonthlyQ[:mMonths] / historical_avg
        scaling_factors_path = os.path.join(rescaled_folder, f"Scenario{j+1}_scaling_factors.txt")
        np.savetxt(scaling_factors_path, scaling_factors, fmt='%.3f')

def CombineRescaledWithRainbase(sites, nScenarios, rescaled_folder, rainbase_data, combined_folder):
    nYears = 30
    mMonths = 12
    total_months = nYears * mMonths
    
    if len(rainbase_data) < total_months:
        rainbase_data = np.tile(rainbase_data, (total_months // len(rainbase_data) + 1))[:total_months]
    
    rainbase_reshaped = rainbase_data.reshape(nYears, mMonths)
    
    for i in range(nScenarios):
        for site in sites:
            rescaled_data = np.loadtxt(os.path.join(rescaled_folder, f"{site}_Scenario{i+1}.csv"), delimiter=',')
            
            if rescaled_data.shape != (nYears, mMonths):
                rescaled_data = rescaled_data.reshape(nYears, mMonths)
            
            combined_data = rescaled_data + rainbase_reshaped
            file_path = os.path.join(combined_folder, f"total_{site}_Scenario{i+1}.csv")
            np.savetxt(file_path, combined_data, fmt='%.3f', delimiter=',')

def CombineSupplyFlows(sites, nScenarios, combined_folder, supply_flows_rescaled_folder):
    nYears = 30
    mMonths = 12
    for i in range(1, nScenarios + 1):
        combined_sum = np.zeros((nYears, mMonths))
        
        for site in sites:
            file_path = os.path.join(combined_folder, f"total_{site}_Scenario{i}.csv")
            data = np.loadtxt(file_path, delimiter=',')
            combined_sum += data
        
        output_file_path = os.path.join(supply_flows_rescaled_folder, f"supply_Scenario{i}.csv")
        np.savetxt(output_file_path, combined_sum, fmt='%.3f', delimiter=',')
        print(f"Saved combined supply flow to {output_file_path}")

def calculate_average_flows(input_folder, output_folder, prefix):
    assure_path_exists(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            data = np.loadtxt(file_path, delimiter=',')
            
            average_data = np.mean(data, axis=0)
            
            output_file_path = os.path.join(output_folder, f"{prefix}_{filename}")
            np.savetxt(output_file_path, average_data.reshape(1, -1), fmt='%.3f', delimiter=',')

def plot_rescaled_combined_flows_against_historical_mean(supply_flows_rescaled_FINAL, historical_data, results_folder):
    # Ensure the results folder exists
    assure_path_exists(results_folder)
    
    # Load historical data and calculate historical mean
    historical_mean = np.mean(historical_data.reshape(-1, 12), axis=0)
    
    # Prepare the figure
    fig, ax = plt.subplots(figsize=(12, 6))
    months = range(1, 13)
    
    # Plot historical mean
    ax.plot(months, historical_mean, label='Historical Mean', color='black', linewidth=2)

    # Plot rescaled combined flows for each scenario
    for file in os.listdir(supply_flows_rescaled_FINAL):
        if file.endswith('.csv'):
            scenario_data = np.loadtxt(os.path.join(supply_flows_rescaled_FINAL, file), delimiter=',')
            scenario_mean = np.mean(scenario_data.reshape(-1, 12), axis=0)
            ax.plot(months, scenario_mean, label=file.split('.')[0], alpha=0.5)

    # Customize plot
    ax.set_title('Rescaled Combined Flows vs Historical Mean')
    ax.set_xlabel('Month')
    ax.set_ylabel('Water Volume')
    ax.set_xticks(months)
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.legend(loc='upper right')

    # Save plot
    plt.tight_layout()
    plot_path = os.path.join(results_folder, 'rescaled_combined_flows_vs_historical_mean.png')
    plt.savefig(plot_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Plot saved to {plot_path}")

import os
import numpy as np
import matplotlib.pyplot as plt

def assure_path_exists(path):
    if not os.path.exists(path):
        print(f"Creating directory: {path}")
        os.makedirs(path)
    else:
        print(f"Directory already exists: {path}")

def plot_individual_rescaled_flows(supply_flows_rescaled_FINAL, historical_data, results_folder, lhs_samples):
    # Ensure the results folder exists
    assure_path_exists(results_folder)
    
    # Load historical data and calculate historical mean
    historical_mean = np.mean(historical_data.reshape(-1, 12), axis=0)
    
    # Prepare the months for x-axis
    months = range(1, 13)
    
    # Plot rescaled combined flows for each scenario individually
    for i, file in enumerate(sorted(os.listdir(supply_flows_rescaled_FINAL))):
        if file.endswith('.csv'):
            scenario_data = np.loadtxt(os.path.join(supply_flows_rescaled_FINAL, file), delimiter=',')
            scenario_mean = np.mean(scenario_data.reshape(-1, 12), axis=0)
            amplitude, lambda_coeff = lhs_samples[i]

            # Prepare the figure
            fig, ax = plt.subplots(figsize=(12, 6))
            
            # Plot historical mean
            ax.plot(months, historical_mean, label='Historical Mean', color='black', linewidth=2)
            
            # Plot scenario mean
            ax.plot(months, scenario_mean, label=f'Rescaled {i+1}', alpha=0.5, color='blue')

            # Customize plot
            ax.set_title(f'Scenario {i+1}: Rescaled Combined Flows vs Historical Mean\nAmplitude: {amplitude:.3f}, Lambda: {lambda_coeff:.3f}')
            ax.set_xlabel('Month')
            ax.set_ylabel('Water Volume')
            ax.set_xticks(months)
            ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
            ax.legend(loc='upper right')

            # Save plot
            plot_path = os.path.join(results_folder, f'scenario_{i+1}_rescaled_combined_flows_vs_historical_mean.png')
            plt.tight_layout()
            plt.savefig(plot_path, bbox_inches='tight', dpi=300)
            plt.close()
            print(f"Plot for Scenario {i+1} saved to {plot_path}")

def main():
    sites = ['All']  
    nMonths = 12
    
    all_param_ranges = [
        [0.47, 1.78],  # Amplitude
        [0, 1]        # lambda
    ]
    
    n_samples = 10

    all_lhs_samples = lhs_params(all_param_ranges, n_samples)
    np.savetxt("LHsamples_all_params.txt", all_lhs_samples, fmt='%.6f')
    
    global supply_lhs_samples
    supply_lhs_samples = all_lhs_samples


    Qgs = np.loadtxt('meltwater_hist_km3.txt')
    Qall = np.loadtxt('total_km3.txt')
    rainbase_data = np.loadtxt('rainbase_km3.txt')

    hist_data = np.loadtxt('meltwater_hist_km3.txt')
    scenario_data = np.loadtxt('total_km3.txt')
    scenario_type = 'Scenario'

    supply_flows_folder = './Supply_flows_only'
    rescaled_folder = os.path.join(supply_flows_folder, '1. rescaled_meltwater')
    combined_folder = os.path.join(supply_flows_folder, '2. combined')
    supply_flows_rescaled_folder = os.path.join(supply_flows_folder, '3. supply_flows_rescaled_combined')
    supply_flows_rescaled_FINAL = os.path.join(supply_flows_folder, '4. supply_flows_rescaled_FINAL')
    results_folder = os.path.join(supply_flows_folder, '6. results')
    individual_plots_folder = os.path.join(supply_flows_folder, '7. individual_scenario_plots')
    
    for folder in [supply_flows_folder, rescaled_folder, combined_folder, supply_flows_rescaled_folder, 
                   supply_flows_rescaled_FINAL, results_folder, individual_plots_folder]:
        assure_path_exists(folder)

    ComponentRescaling(sites, Qgs, supply_lhs_samples, rescaled_folder)
    CombineRescaledWithRainbase(sites, n_samples, rescaled_folder, rainbase_data, combined_folder)
    CombineSupplyFlows(sites, n_samples, combined_folder, supply_flows_rescaled_folder)
    calculate_average_flows(supply_flows_rescaled_folder, supply_flows_rescaled_FINAL, "FINAL_combined_rescaled")

    plot_individual_rescaled_flows(supply_flows_rescaled_FINAL,Qall, individual_plots_folder, supply_lhs_samples)

    print("Process completed successfully!")

if __name__ == "__main__":
    main()
